In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-model-features/modelWI_3.pkl
/kaggle/input/m5-model-features/__results__.html
/kaggle/input/m5-model-features/modelTX_3.pkl
/kaggle/input/m5-model-features/modelCA_3.pkl
/kaggle/input/m5-model-features/modelWI_1.pkl
/kaggle/input/m5-model-features/modelCA_2.pkl
/kaggle/input/m5-model-features/submission.csv
/kaggle/input/m5-model-features/__notebook__.ipynb
/kaggle/input/m5-model-features/dict.pkl
/kaggle/input/m5-model-features/__output__.json
/kaggle/input/m5-model-features/modelCA_1.pkl
/kaggle/input/m5-model-features/modelCA_4.pkl
/kaggle/input/m5-model-features/modelTX_2.pkl
/kaggle/input/m5-model-features/modelTX_1.pkl
/kaggle/input/m5-model-features/modelWI_2.pkl
/kaggle/input/m5-model-features/processed_pickle.pkl
/kaggle/input/m5-model-features/custom.css
/kaggle/input/m5-model-features/__results___files/__results___52_1.png
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasti

In [2]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import lightgbm
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pickle
import gc
import optuna
from optuna.integration import LightGBMPruningCallback
import time

In [3]:
data=pickle.load(open('/kaggle/input/m5-model-features/processed_pickle.pkl','rb'))

In [4]:
cat_column=[]
for i in data.columns:
    if(str(data.dtypes[i])=='category'):
        cat_column.append(i)

In [5]:
cat_column

['event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'event1_lag_-2',
 'event1_lag_-1',
 'event1_lag_1',
 'event1_lag_2']

In [6]:
gc.collect()

129

In [7]:
data=data[data['d']>=1970-(365*2)+(3*28)]

In [8]:
data['d'].unique()[-1]

1969

In [9]:
1969-28-28-28
#1885

1885

In [10]:
X=data[data['d']<1885].drop('sold',axis=1)
y=data[data['d']<1885]['sold']

x_validate=data[(data['d']>=1885)&(data['d']<(1885+(28*2)))].drop('sold',axis=1)
y_validate=data[(data['d']>=1885)&(data['d']<(1885+(28*2)))]['sold']

In [11]:
del data
gc.collect()

21

In [12]:
!pip install verstack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.6 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.6 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.6 MB 487.0 kB/s eta 0:00:20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.6 MB 487.0 kB/s eta 0:00:20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/9.6 MB 264.9 kB/s eta 0:00:37

...

      Successfully uninstalled optuna-3.1.0


  Attempting uninstall: category_encoders
    Found existing installation: category-encoders 2.6.0


    Uninstalling category-encoders-2.6.0:


      Successfully uninstalled category-encoders-2.6.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
prophet 1.1.1 requires holidays>=0.14.2, but you have holidays 0.11.3.1 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
featuretools 1.11.1 requires holidays>=0.13, but you have holidays 0.11.3.1 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [13]:
from verstack import LGBMTuner
tuner = LGBMTuner(metric = 'rmse') # <- the only required argument
tuner.fit(X,y)


 * Initiating LGBMTuner.fit
     . Settings:
     .. Trying 100 trials
     .. Evaluation metric: rmse 
     .. Study direction: minimize rmse



     . Trial number: 0 finished
     .. Optimization score (lower-better): rmse: 1.9843668457132044
 ...........................................................................


     . Trial number: 1 finished
     .. Optimization score (lower-better): rmse: 1.9798582698420217
 ...........................................................................


     . Trial number: 2 finished
     .. Optimization score (lower-better): rmse: 2.0433456883338996
 ...........................................................................


     . Trial number: 3 finished
     .. Optimization score (lower-better): rmse: 2.038833338727186
 ...........................................................................


...

     . Trial number: 17 finished
     .. Optimization score (lower-better): rmse: 1.9726023208801513
 ...........................................................................


     . Trial number: 21 finished
     .. Optimization score (lower-better): rmse: 1.9798214932712086
 ...........................................................................


     . Trial number: 22 finished
     .. Optimization score (lower-better): rmse: 1.9685280856212015
 ...........................................................................


     . Trial number: 31 finished
     .. Optimization score (lower-better): rmse: 1.9620781997003756
 ...........................................................................


     . Trial number: 33 finished
     .. Optimization score (lower-better): rmse: 1.9791428783780198
 ...........................................................................


In [ ]:
tuner.best_params


In [ ]:
import pickle
pickle.dump(tuner,open('tuner.pkl','wb'))